In [71]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import time
%matplotlib inline
from bs4 import BeautifulSoup as bs
import requests
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sqlalchemy import *
from sqlalchemy import create_engine
import calendar
import sqlite3 as sql
pd.options.display.float_format = '{:,.4f}'.format

init_notebook_mode(connected=True)

import os
main_dir = os.getcwd()

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from alphavantage import *
from yahoo_query import *
from option_slam_earnings import *
from reuters_query import reuters_query, reuters_insiders
from alphaquery import alphaquery

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Options')
from optionsFunctions import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Earnings')
from yahoo_earnings import *
from lookup_earnings import lookup_earnings

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Processing')
from hv_calc import *

os.chdir(main_dir)

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir(dbs_dir)

earn_engine = create_engine('sqlite:///earningsHistory.db', echo=False)
yahoo_engine = create_engine('sqlite:///yahoo.db', echo = False)
reuters_engine = create_engine('sqlite:///reuters.db', echo=False)

In [93]:
tickers = ['CMC', 'AZZ', 'PSMT', 'BBBY', 'STZ', 'MSM', 'DAL', 'LEN', 'AYI', 'GBX']

annual_statements = {}
quarterly_statements = {}
profiles = []
query = 'SELECT * FROM {} WHERE Underlying IN ' + str(tuple(tickers))

for yahoo_table in yahoo_engine.table_names():
    if 'BalanceSheet' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates().fillna(np.nan)
        if 'annual' in yahoo_table:
            curr_table.year = pd.to_datetime(curr_table.year)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            annual_statements['balanceSheet'] = curr_table
        else:
            curr_table.quarter = pd.to_datetime(curr_table.quarter)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            quarterly_statements['balanceSheet'] = curr_table
            
    elif 'IncomeStatement' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates().fillna(np.nan)
        if 'annual' in yahoo_table:
            curr_table.year = pd.to_datetime(curr_table.year)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            annual_statements['incomeStatements'] = curr_table
        else:
            curr_table.quarter = pd.to_datetime(curr_table.quarter)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            quarterly_statements['incomeStatements'] = curr_table
            
    elif 'CashFlow' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates().fillna(np.nan)
        if 'annual' in yahoo_table:
            curr_table.year = pd.to_datetime(curr_table.year)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            annual_statements['cashFlows'] = curr_table
        else:
            curr_table.quarter = pd.to_datetime(curr_table.quarter)
            
            for col in curr_table.columns:
                if isinstance(curr_table[col][0], dt.datetime) == False and col != 'maxAge':
                    curr_table[col] = pd.to_numeric(curr_table[col])
                    
            quarterly_statements['cashFlows'] = curr_table
    elif 'profiles' in yahoo_table:
        curr_table = pd.read_sql_query(query.format(yahoo_table), con = yahoo_engine,
                                       index_col = 'Underlying').drop_duplicates()[['industry', 'sector']]
        profiles.append(curr_table)
    else:
        continue
profiles = pd.concat(profiles, axis = 0)

In [129]:
ticker = 'GBX'

annual_statements['cashFlows'][annual_statements['cashFlows'].index == ticker].T

Underlying,GBX,GBX,GBX,GBX
year,2015-08-31 00:00:00,2016-08-31 00:00:00,2017-08-31 00:00:00,2018-08-31 00:00:00
capitalExpenditures,-105989000,-139013000,-86065000,-176848000
changeInCash,-11986000,49749000,388787000,-80811000
changeToAccountReceivables,"13,652,000.0000","-32,051,000.0000","-25,272,000.0000","-83,551,000.0000"
changeToInventory,-143849000,53711000,-2787000,-26592000
changeToLiabilities,85727000,-35216000,7617000,33801000
changeToNetincome,88854000,102003000,63937000,-30717000
changeToOperatingActivities,"-90,039,000.0000","2,165,000.0000","58,573,000.0000","-19,908,000.0000"
depreciation,45156000,63345000,65129000,74356000
dividendsPaid,-16491000,-23303000,-24890000,-29914000


In [130]:
quarterly_statements['cashFlows'][quarterly_statements['cashFlows'].index == ticker].T

Underlying,GBX,GBX,GBX,GBX
quarter,2017-11-30 00:00:00,2018-02-28 00:00:00,2018-05-31 00:00:00,2018-08-31 00:00:00
capitalExpenditures,"-29,893,000.0000","-23,610,000.0000","-65,153,000.0000","-58,192,000.0000"
changeInCash,"-20,060,000.0000","-5,398,000.0000","3,961,000.0000","-59,314,000.0000"
changeToAccountReceivables,"-35,510,000.0000","9,829,000.0000","701,000.0000","-58,571,000.0000"
changeToInventory,"-16,311,000.0000","6,100,000.0000","5,941,000.0000","-22,322,000.0000"
changeToLiabilities,8128000,-4283000,7216000,22740000
changeToNetincome,"-12,371,000.0000","-25,949,000.0000","-3,536,000.0000","11,139,000.0000"
changeToOperatingActivities,"-29,237,000.0000","-34,458,000.0000","24,250,000.0000","19,537,000.0000"
depreciation,"18,370,000.0000","18,084,000.0000","18,707,000.0000","19,195,000.0000"
dividendsPaid,"-319,000.0000","-13,227,000.0000","-8,320,000.0000","-8,048,000.0000"


In [131]:
annual_statements['incomeStatements'][annual_statements['incomeStatements'].index == ticker].T

Underlying,GBX,GBX,GBX,GBX
year,2018-08-31 00:00:00,2017-08-31 00:00:00,2016-08-31 00:00:00,2015-08-31 00:00:00
costOfRevenue,2110409000,1747865000,2128087000,2067925000
discontinuedOperations,NaN,NaN,NaN,NaN
ebit,208616000,250692000,392756000,385562000
effectOfAccountingCharges,NaN,NaN,NaN,NaN
extraordinaryItems,NaN,NaN,NaN,NaN
grossProfit,409055000,421299000,551437000,537353000
incomeBeforeTax,"204,956,000.0000","224,476,000.0000","397,146,000.0000","377,469,000.0000"
incomeTaxExpense,32893000,64014000,112322000,112160000
interestExpense,-30946000,-23519000,-17268000,-18975000


In [132]:
quarterly_statements['incomeStatements'][quarterly_statements['incomeStatements'].index == ticker].T

Underlying,GBX,GBX,GBX,GBX
quarter,2018-08-31 00:00:00,2018-05-31 00:00:00,2018-02-28 00:00:00,2017-11-30 00:00:00
costOfRevenue,583319000,532880000,523989000,470221000
discontinuedOperations,NaN,NaN,NaN,NaN
ebit,54578000,56714000,55053000,42271000
effectOfAccountingCharges,NaN,NaN,NaN,NaN
extraordinaryItems,NaN,NaN,NaN,NaN
grossProfit,105887000,108507000,105347000,89314000
incomeBeforeTax,"47,273,000.0000","52,183,000.0000","53,988,000.0000","51,512,000.0000"
incomeTaxExpense,10115000,15944000,-11301000,18135000
interestExpense,-7694000,-7392000,-7895000,-7964000


In [137]:
annual_statements['balanceSheet'][annual_statements['balanceSheet'].index == ticker].T

Underlying,GBX,GBX,GBX,GBX
year,2018-08-31 00:00:00,2017-08-31 00:00:00,2016-08-31 00:00:00,2015-08-31 00:00:00
accountsPayable,226405000,180592000,182334000,263665000
capitalSurplus,442569000,315306000,282886000,295444000
cash,530655000,611466000,222679000,172930000
commonStock,NaN,NaN,NaN,NaN
deferredLongTermAssetCharges,NaN,NaN,NaN,NaN
deferredLongTermLiab,"1,824,000.0000","2,623,000.0000","3,481,000.0000","3,823,000.0000"
goodWill,"78,211,000.0000","68,590,000.0000","43,265,000.0000","43,265,000.0000"
intangibleAssets,"43,960,000.0000","40,621,000.0000","29,015,000.0000","32,408,000.0000"
inventory,432314000,400127000,365805000,445535000


In [138]:
quarterly_statements['balanceSheet'][quarterly_statements['balanceSheet'].index == ticker].T

Underlying,GBX,GBX,GBX,GBX
quarter,2018-08-31 00:00:00,2018-05-31 00:00:00,2018-02-28 00:00:00,2017-11-30 00:00:00
accountsPayable,226405000,204658000,215491000,193919000
capitalSurplus,442569000,438560000,316073000,312789000
cash,530655000,589969000,586008000,591406000
commonStock,NaN,NaN,NaN,NaN
deferredLongTermAssetCharges,NaN,NaN,NaN,NaN
deferredLongTermLiab,"1,824,000.0000","2,002,000.0000","2,209,000.0000","2,416,000.0000"
goodWill,78211000,70347000,69011000,67783000
intangibleAssets,"43,960,000.0000","36,122,000.0000","38,063,000.0000","39,056,000.0000"
inventory,432314000,396518000,408419000,411371000


In [128]:
for key,val in annual_statements.items():
    print(key)
    print(val.columns)

balanceSheet
Index(['year', 'accountsPayable', 'capitalSurplus', 'cash', 'commonStock',
       'deferredLongTermAssetCharges', 'deferredLongTermLiab', 'goodWill',
       'intangibleAssets', 'inventory', 'longTermDebt', 'longTermInvestments',
       'maxAge', 'minorityInterest', 'netReceivables', 'netTangibleAssets',
       'otherAssets', 'otherCurrentAssets', 'otherCurrentLiab', 'otherLiab',
       'otherStockholderEquity', 'propertyPlantEquipment', 'retainedEarnings',
       'shortLongTermDebt', 'shortTermInvestments', 'totalAssets',
       'totalCurrentAssets', 'totalCurrentLiabilities', 'totalLiab',
       'totalStockholderEquity', 'treasuryStock'],
      dtype='object')
cashFlows
Index(['year', 'capitalExpenditures', 'changeInCash',
       'changeToAccountReceivables', 'changeToInventory',
       'changeToLiabilities', 'changeToNetincome',
       'changeToOperatingActivities', 'depreciation', 'dividendsPaid',
       'effectOfExchangeRate', 'investments', 'issuanceOfStock',
       '